<a href="https://colab.research.google.com/github/Gabriel-Ax/Test/blob/main/Copy_of_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [1]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [2]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [3]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Quando é a próxima Imersão IA com Google Gemini da Alura?"
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no Instagram, LinkedIn, Twitter e Facebook. Eles costumam anunciar novos cursos e imersões por lá.
*   **Inscrever-se na newsletter da Alura:** Ao se inscrever, você receberá e-mails com novidades, promoções e anúncios de novos cursos, incluindo as Imersões.
*   **Visitar a página da Imersão IA com Google Gemini no site da Alura:** Mesmo que não tenha uma data agendada, a página pode ter informações sobre edições anteriores e a possibilidade de se inscrever para receber novidades sobre a próxima turma.

In [4]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools":[{"google_search": {}}]}
    # Inserir a tool de busca do Google ###############################################
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio. A Imersão IA – Google Gemini é um curso 100% online e gratuito, realizado em parceria entre o Google e a Alura. O curso ensina a usar a IA do Google Gemini na prática, mesmo para profissionais de outras áreas, com aulas sobre automação de tarefas e desenvolvimento de agentes autônomos.


In [5]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini da Alura']
Páginas utilizadas na resposta: starten.tech, youtube.com



In [6]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install google-adk

In [7]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [21]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [22]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [29]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca notícias sobre o tópico indicado",
      tools=[google_search],
      instruction="""Você é um assistente de pesquisa, e a sua tarefa é usar a ferramenta de pesquisa do google (google_search).
        Para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações ensiasmadas, é possivel que ele não seja tão relevante assim
        e pode ser substituído o outro que tenha mais.
        Esses lançamentos devem ser atuais, de no máximo um mês antes da data de hoje.
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  plano_do_post = call_agent(buscador, entrada_do_agente_buscador)
  return plano_do_post

In [13]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, data_de_hoje):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        description="Agente que planeja posts",
        tools=[google_search],
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais, Com base na lista de lançamentos mais recentes e relevantes buscador,
        você deve usar a ferramenta de busca do Google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderiamos abordar em um post sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais informações osbre os temas e aprofundar.
        Ao Final, você ira escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [17]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [18]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais.",
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [32]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:

  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n --------- Assunto do Agente Buscador ----- \n")
  display(to_markdown(lancamentos_buscados))
  print("------------------------------------------------")

  planejamento = agente_planejador(topico, data_de_hoje)
  print("\n --------- Assunto do Agente Planejador ----- \n")
  display(to_markdown(planejamento))
  print("------------------------------------------------")

  rascunho = agente_redator(topico, planejamento)
  print("\n --------- Assunto do Agente Redator ----- \n")
  display(to_markdown(rascunho))
  print("------------------------------------------------")

  texto_revisado = agente_revisor(topico, rascunho)
  print("\n --------- Assunto do Agente Revisor ----- \n")
  display(to_markdown(texto_revisado))
  print("------------------------------------------------")

  print(to_markdown(texto_revisado))

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Pizzarias e Tecnologias
Maravilha! Vamos então criar o post sobre novidades em Pizzarias e Tecnologias

 --------- Assunto do Agente Buscador ----- 



> Para fornecer as informações mais recentes sobre pizzarias e tecnologias, farei algumas pesquisas no Google. Aqui estão as perguntas que usarei:
> 
> 
> Com certeza! Aqui estão os 5 lançamentos mais relevantes sobre o tema "Pizzarias e Tecnologias" para maio de 2025:
> 
> 1.  **Automação e Inteligência Artificial na Produção de Pizzas:** Robôs e máquinas inteligentes estão sendo usados para preparar pizzas de forma rápida e precisa, modelando massas e aplicando ingredientes automaticamente. A inteligência artificial (IA) otimiza o inventário, monitora o uso de ingredientes e prevê a demanda, evitando desperdícios. Sistemas de gerenciamento de pedidos também estão sendo automatizados para melhorar a eficiência.
> 2.  **Sustentabilidade e Tecnologia Verde:** Pizzarias estão adotando soluções que combinam tecnologia e práticas ecológicas, como equipamentos energeticamente eficientes e embalagens biodegradáveis.
> 3.  **Integração com Apps de Delivery e Sistemas PDV:** A integração com aplicativos de delivery é uma tendência importante, com sistemas de gestão que oferecem funções como integração com delivery, pagamentos digitais e relatórios em tempo real. Os sistemas PDV (Ponto de Venda) também estão evoluindo, permitindo que as pizzarias tomem decisões baseadas em dados, reduzindo erros e otimizando processos.
> 4.  **Hiperpersonalização:** A hiperpersonalização possibilita que os consumidores integrem suas experiências através de mecanismos digitais ou ações físicas no restaurante, como menus personalizados e programas de fidelidade.
> 5.  **Realidade Aumentada e Virtual no Atendimento:** A realidade aumentada e virtual estão sendo utilizadas para melhorar a experiência do cliente no atendimento.
> 
> Além disso, a Expo Pizzaria e Burger Expo 2025, que acontecerá em julho, reunirá expositores de renome nacional e internacional, trazendo as marcas mais inovadoras do setor. O evento oferecerá palestras e workshops sobre tendências de consumo, inovações tecnológicas, marketing, gestão de qualidade e sustentabilidade.


------------------------------------------------

 --------- Assunto do Agente Planejador ----- 



> Ok, sou um planejador de conteúdo especializado em redes sociais e vou criar um plano de conteúdo sobre pizzarias e tecnologias com base nas informações mais recentes e relevantes, utilizando a ferramenta de busca do Google.
> 
> 
> Com base nas minhas pesquisas, o tema mais relevante para um post sobre "Pizzarias e Tecnologias" em 2025 é:
> 
> **Tema:** **O Futuro da Pizzaria: Tendências Tecnológicas para 2025**
> 
> **Pontos Relevantes:**
> 
> *   **Automação e IA:**
>     *   Automação na produção para agilizar e padronizar o preparo.
>     *   IA para análise preditiva, personalização de ofertas e chatbots inteligentes.
>     *   Otimização do estoque e gestão de custos com automação.
> *   **Experiência do Cliente Hiperpersonalizada:**
>     *   Menus personalizados e programas de fidelidade.
>     *   Integração de experiências digitais e físicas no restaurante.
> *   **Delivery e Atendimento:**
>     *   Integração com apps de delivery para gestão centralizada e rastreamento de pedidos.
>     *   Web Delivery: integrar o cardápio ao site da pizzaria.
>     *   Alternativas para reduzir custos com comissões de plataformas de delivery.
>     *   Otimização do fluxo de trabalho na cozinha com KDS (Kitchen Display System).
> *   **Marketing Digital:**
>     *   Uso do Instagram para mostrar os pratos e interagir com clientes.
>     *   Otimização do Google Meu Negócio para ser encontrado online.
>     *   Estratégias de marketing focadas em fidelização e promoções.
> *   **Sistemas de Gestão:**
>     *   Sistemas PDV (Ponto de Venda) com integração de delivery, pagamentos digitais e relatórios em tempo real.
>     *   Importância de escolher um sistema de gestão que atenda às necessidades da pizzaria, seja fácil de usar, ofereça mobilidade, integração e suporte.
> *   **Novas tecnologias:**
>     *   Comanda eletrônica.
>     *   QR code para cardápio digital e pagamento.
> 
> **Plano de Assuntos para o Post:**
> 
> 1.  **Introdução:** A importância da tecnologia para pizzarias em 2025.
> 2.  **Automação e Inteligência Artificial:**
>     *   Como a automação está transformando a produção de pizzas.
>     *   Exemplos de uso de IA para melhorar a experiência do cliente e a gestão.
> 3.  **Hiperpersonalização:**
>     *   Como oferecer um menu personalizado e programas de fidelidade.
>     *   A importância de integrar o online e o offline para uma experiência completa.
> 4.  **Delivery e Atendimento:**
>     *   Dicas para otimizar o delivery e reduzir custos com plataformas.
>     *   Como o KDS pode melhorar a eficiência da cozinha e a satisfação do cliente.
> 5.  **Marketing Digital:**
>     *   Estratégias para usar o Instagram e o Google Meu Negócio a favor da sua pizzaria.
>     *   A importância de construir uma boa reputação online.
> 6.  **Sistemas de Gestão:**
>     *   Como escolher o sistema de gestão ideal para sua pizzaria.
>     *   Funcionalidades essenciais para otimizar a operação.
> 7.  **Exemplos de Aplicativos:**
>     *   Exemplos de aplicativos para pedir e otimizar sua pizzaria.
> 8.  **Conclusão:** O futuro promissor das pizzarias que investem em tecnologia.
> 
> Este plano de conteúdo aborda as principais tendências e tecnologias que podem impulsionar o sucesso de uma pizzaria em 2025, oferecendo informações valiosas e práticas para os proprietários de pizzarias se manterem competitivos e inovadores.
> 


------------------------------------------------

 --------- Assunto do Agente Redator ----- 



> 🍕🍕Já pediu sua pizza hoje? Mas você já pensou em quanta tecnologia está por trás da sua redonda favorita?
> 
> Em 2025, as pizzarias estão com tudo no mundo da tecnologia! 🚀
> 
> 🤖 Automação? Inteligência Artificial? Elas estão na produção, atendimento e até na personalização do seu pedido! Já pensou em um menu feito sob medida pra você? 🤤
> 
> 📱 Delivery otimizado, marketing digital que te encontra no Insta e sistemas de gestão que fazem tudo rodar liso. 
> 
> Quer saber mais sobre o futuro das pizzarias e como a tecnologia tá mudando tudo? Fique de olho nos próximos posts da Alura! 😉
> 
> #PizzaTech #FuturoDaPizza #Alura #InovaçãoNaCozinha
> 


------------------------------------------------

 --------- Assunto do Agente Revisor ----- 



> O rascunho está ótimo, mas podemos fazer alguns ajustes para deixá-lo ainda mais interessante para o público do Instagram:
> 
> **Problemas e Sugestões:**
> 
> *   **Emojis:** O uso de emojis está bom, mas podemos adicionar mais alguns para deixar o post mais atrativo visualmente.
> *   **Chamada para Ação:** A chamada para ação no final ("Fique de olho nos próximos posts da Alura!") é boa, mas podemos torná-la mais direta e envolvente.
> *   **Hashtags:** As hashtags estão relevantes, mas podemos adicionar algumas mais específicas para aumentar o alcance do post.
> 
> **Rascunho Revisado:**
> 
> 🍕🍕 Pediu pizza hoje? 🤔 Sabia que a sua redonda favorita tem um monte de tecnologia por trás? 🤯
> 
> Em 2025, as pizzarias estão DOMINANDO o mundo tech! 🚀
> 
> 🤖 Automação, Inteligência Artificial... Elas estão na produção, no atendimento e até personalizando seu pedido! 🤩 Já imaginou um menu feito especialmente pra VOCÊ? 🤤
> 
> 📱 Delivery super rápido, marketing digital que te acha no Insta e sistemas que fazem tudo funcionar perfeitamente. ✨
> 
> Curioso pra saber mais sobre o futuro das pizzarias e como a tecnologia está transformando tudo? 🍕💻 Fique ligado nos próximos posts da Alura e #VemProFuturo! 😉
> 
> #PizzaTech #FuturoDaPizza #Alura #InovaçãoNaCozinha #TechFood #PizzariaDoFuturo


------------------------------------------------
<IPython.core.display.Markdown object>
